In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-6qnjq1z4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-6qnjq1z4
  Resolved https://github.com/huggingface/transformers to commit 083e36923a19650fa264c4173db2f63ab124bb27
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8075921 sha256=dc09b046ff74c4f944515df85497422b8dc48bf656afa8d8ce663230d34fd836
  Stored in directory: /tmp/pip-ephem-wheel-cache-vz6dirut/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="test", use_auth_token=True)
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])


/usr/local/lib/python3.10/dist-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 7308it [00:00, 55878.18it/s]


...


Reading metadata...: 5069it [00:00, 78833.81it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 5699it [00:00, 76659.46it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1346it [00:00, 50923.05it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [ ]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)


Map (num_proc=2):   0%|          | 0/12360 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/5069 [00:00<?, ? examples/s]

In [ ]:
common_voice.save_to_disk("common_voice")

Saving the dataset (0/24 shards):   0%|          | 0/12360 [00:00<?, ? examples/s]

Saving the dataset (0/10 shards):   0%|          | 0/5069 [00:00<?, ? examples/s]

In [ ]:
!du -sh './common_voice/'


16G	./common_voice/


In [ ]:
!zip -r model.zip './common_voice/'


  adding: common_voice/ (stored 0%)
  adding: common_voice/test/ (stored 0%)
  adding: common_voice/test/data-00006-of-00010.arrow (deflated 90%)
  adding: common_voice/test/data-00001-of-00010.arrow (deflated 90%)
  adding: common_voice/test/dataset_info.json (deflated 51%)
  adding: common_voice/test/data-00007-of-00010.arrow (deflated 91%)
  adding: common_voice/test/data-00004-of-00010.arrow (deflated 90%)
  adding: common_voice/test/data-00005-of-00010.arrow (deflated 89%)
  adding: common_voice/test/data-00002-of-00010.arrow (deflated 89%)
  adding: common_voice/test/data-00009-of-00010.arrow (deflated 91%)
  adding: common_voice/test/state.json (deflated 76%)
  adding: common_voice/test/data-00008-of-00010.arrow (deflated 90%)
  adding: common_voice/test/data-00000-of-00010.arrow (deflated 89%)
  adding: common_voice/test/data-00003-of-00010.arrow (deflated 89%)
  adding: common_voice/train/ (stored 0%)
  adding: common_voice/train/data-00002-of-00024.arrow (deflated 91%)
  addi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp 'model.zip' '/content/drive/My Drive/'
